# Helm 201

A deep-dive into Helm (v3) and details like

* Templating
* Charts and Subcharts
* Usage and internal structure in Kubernetes
* Integrations

In [61]:
wd_init = "work/helm-init"



In [2]:
!helm version


version.BuildInfo{Version:"v3.1.3+2.el8", GitCommit:"4edcae3c96edf7fa6a3bcf93de6d85763aed1284", GitTreeState:"clean", GoVersion:"go1.13.4"}


---
---

## Init

* Create a new template / Helm Chart

In [3]:
!echo $wd_init
!mkdir -p $wd_init

!helm create $wd_init/demo-helm-201

!tree $wd_init

work/helm-init
Creating work/helm-init/demo-helm-201
work/helm-init
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    ├── templates
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── ingress.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    └── values.yaml

4 directories, 9 files


## Chart Structure and Overview

* `Chart.yaml` common meta information
  * avoid using `appVersion` - handle version in env-specific value-files or as field which will be overwritten during helm execution

In [10]:
!cat $wd_init/demo-helm-201/Chart.yaml | grep -B2 -i 'version:'

apiVersion: v2
--
# This is the chart version. This version number should be incremented each time you make changes
# to the chart and its templates, including the app version.
version: 0.1.0
--
# This is the version number of the application being deployed. This version number should be
# incremented each time you make changes to the application.
appVersion: 1.16.0


----
* Templating
  * include output from namespaced functions
  * Whitespaces and new lines, indent


* Action (00)
  * modify service template

In [30]:
!echo "Render template and generate Kubernetes resource files"

!helm template release-name $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.yaml --output-dir=work/out/common

Render template and generate Kubernetes resource files
wrote work/out/common/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/common/demo-helm-201/templates/service.yaml
wrote work/out/common/demo-helm-201/templates/deployment.yaml

---
# Source: demo-helm-201/templates/tests/test-connection.yaml
apiVersion: v1
kind: Pod
metadata:
  name: "release-name-demo-helm-201-test-connection"
  labels:
    helm.sh/chart: demo-helm-201-0.1.0
    app.kubernetes.io/name: demo-helm-201
    app.kubernetes.io/instance: release-name
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
  annotations:
    "helm.sh/hook": test-success
spec:
  containers:
    - name: wget
      image: busybox
      command: ['wget']
      args: ['release-name-demo-helm-201:80']
  restartPolicy: Never


----
* Templating, Variables
  * `.Values.*` holds all variables (from file and command line)
  * `values.yaml` and any additional values-file will be merged

 


* Action (01)
  * `image.version`
  * env-specific values file


In [33]:
!echo "Render template and generate Kubernetes resource files for *TEST* stage"

!helm template release-name-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

Render template and generate Kubernetes resource files for *TEST* stage
wrote work/out/test/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/test/demo-helm-201/templates/service.yaml
wrote work/out/test/demo-helm-201/templates/deployment.yaml

---
# Source: demo-helm-201/templates/tests/test-connection.yaml
apiVersion: v1
kind: Pod
metadata:
  name: "release-name-test-demo-helm-201-test-connection"
  labels:
    helm.sh/chart: demo-helm-201-0.1.0
    app.kubernetes.io/name: demo-helm-201
    app.kubernetes.io/instance: release-name-test
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
  annotations:
    "helm.sh/hook": test-success
spec:
  containers:
    - name: wget
      image: busybox
      command: ['wget']
      args: ['release-name-test-demo-helm-201:8001']
  restartPolicy: Never


---

* Templating, Functions
  * `_helpers.tpl` holds set of helper functions used in template files
  * common Go template functions are included (and, or, len, ...)

----
* Templating, Functions
  * `with` set the scope
  * `range` iterate

* Action (02)
  * new `sa.yaml`
  * values in `values.test.yaml`

In [36]:
!echo "Render template and generate Kubernetes resource files for *TEST* stage"

!helm template release-name-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

Render template and generate Kubernetes resource files for *TEST* stage
wrote work/out/test/demo-helm-201/templates/sa.yaml
wrote work/out/test/demo-helm-201/templates/sa.yaml
wrote work/out/test/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/test/demo-helm-201/templates/service.yaml
wrote work/out/test/demo-helm-201/templates/deployment.yaml

---
# Source: demo-helm-201/templates/tests/test-connection.yaml
apiVersion: v1
kind: Pod
metadata:
  name: "release-name-test-demo-helm-201-test-connection"
  labels:
    helm.sh/chart: demo-helm-201-0.1.0
    app.kubernetes.io/name: demo-helm-201
    app.kubernetes.io/instance: release-name-test
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
  annotations:
    "helm.sh/hook": test-success
spec:
  containers:
    - name: wget
      image: busybox
      command: ['wget']
      args: ['release-name-test-demo-helm-201:8001']
  restartPolicy: Never


----
* Charts and Subcharts
  * Subchart is a standalone chart
  * only parent knows the subcharts / childrens
  * ...and only parent can override fields

* Action 
  * checkout existing chart
  * create a new chart in `parent/chart` directory
  * configure dependency

In [37]:
!echo "Print out existing chart (without subchart)"

!tree $wd_init

Print out existing chart
work/helm-init
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    ├── templates
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── ingress.yaml
    │   ├── sa.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    ├── values.test.yaml
    └── values.yaml

4 directories, 11 files


In [40]:
!echo "Create new (sub)chart in the *charts* dir"

!helm create $wd_init/demo-helm-201/charts/demo-subchart
!rm -rf $wd_init/demo-helm-201/charts/demo-subchart/templates/*

!tree $wd_init

Create new (sub)chart in the *charts* dir
Creating work/helm-init/demo-helm-201/charts/demo-subchart
work/helm-init
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    │   └── demo-subchart
    │       ├── Chart.yaml
    │       ├── charts
    │       ├── templates
    │       └── values.yaml
    ├── templates
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── ingress.yaml
    │   ├── sa.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    ├── values.test.yaml
    └── values.yaml

7 directories, 13 files


* Action (03)
  * template for `ConfigMap` in subchart
  * value file in subchart
  * override options

In [43]:
!echo "Dry-Run subchart - with local value file"

!helm install --generate-name --dry-run --debug $wd_init/demo-helm-201/charts/demo-subchart

Dry-Run subchart - with local value file
install.go:158: [debug] Original chart version: ""
install.go:175: [debug] CHART PATH: /Users/haddouti/repos/haf-tech/helm-201/work/helm-init/demo-helm-201/charts/demo-subchart

NAME: demo-subchart-1644082592
LAST DEPLOYED: Sat Feb  5 18:36:40 2022
NAMESPACE: demo-helm
STATUS: pending-install
REVISION: 1
TEST SUITE: None
USER-SUPPLIED VALUES:
{}

COMPUTED VALUES:
commonItem: mega-one

HOOKS:
MANIFEST:
---
# Source: demo-subchart/templates/03_configmap.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: demo-subchart-1644082592-cm
data:
  item: mega-one



In [46]:
!echo "Render entire chart - with *pre* value file, to override subchart fields"

!helm template release-name-pre $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.pre.yaml --output-dir=work/out/pre

Dry-Run entire chart - with *pre* value file, to override subchart fields
wrote work/out/pre/demo-helm-201/templates/sa.yaml
wrote work/out/pre/demo-helm-201/templates/sa.yaml
wrote work/out/pre/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/pre/demo-helm-201/charts/demo-subchart/templates/03_configmap.yaml
wrote work/out/pre/demo-helm-201/templates/service.yaml
wrote work/out/pre/demo-helm-201/templates/deployment.yaml

---
# Source: demo-helm-201/templates/tests/test-connection.yaml
apiVersion: v1
kind: Pod
metadata:
  name: "release-name-pre-demo-helm-201-test-connection"
  labels:
    helm.sh/chart: demo-helm-201-0.1.0
    app.kubernetes.io/name: demo-helm-201
    app.kubernetes.io/instance: release-name-pre
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
  annotations:
    "helm.sh/hook": test-success
spec:
  containers:
    - name: wget
      image: busybox
      command: ['wget']
      args: ['release-name-pre-demo-helm-201:8001']
  res

In [47]:
!echo "Render entire chart - with *test* value file, to *NOT* override subchart fields"

!helm template release-name-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

Render entire chart - with *test* value file, to *NOT* override subchart fields
wrote work/out/test/demo-helm-201/templates/sa.yaml
wrote work/out/test/demo-helm-201/templates/sa.yaml
wrote work/out/test/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/test/demo-helm-201/charts/demo-subchart/templates/03_configmap.yaml
wrote work/out/test/demo-helm-201/templates/service.yaml
wrote work/out/test/demo-helm-201/templates/deployment.yaml

---
# Source: demo-helm-201/templates/tests/test-connection.yaml
apiVersion: v1
kind: Pod
metadata:
  name: "release-name-test-demo-helm-201-test-connection"
  labels:
    helm.sh/chart: demo-helm-201-0.1.0
    app.kubernetes.io/name: demo-helm-201
    app.kubernetes.io/instance: release-name-test
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
  annotations:
    "helm.sh/hook": test-success
spec:
  containers:
    - name: wget
      image: busybox
      command: ['wget']
      args: ['release-name-test-demo-helm-2

---
---

## Life Cycle and Hooks

Hooks allows to execute specific resource definitions at defined points in the Helm life cycle.
Available hooks

* `pre-install`
* `post-install`
* `pre-delete`
* `post-delete`
* `pre-upgrade`
* `post-upgrade`
* `pre-rollback`
* `post-rollback`
* `test`

Examples

* prepare the installation and create specific resources beforehand (ConfigMap, Job etc)
* clean up database before uninstalling application


### Pre-Install and Post-Delete

Example for `pre-install` and `post-delete` hook. Hooks are usual Kubernetes resource definitions with special annotations.

```
metadata:
  annotations:
    "helm.sh/hook": pre-install, post-delete
    "helm.sh/hook-weight": "-5"
    "helm.sh/hook-delete-policy": hook-succeeded
```

In [50]:
!echo "Create new (sub)chart in the *charts* dir"

!helm create $wd_init/demo-helm-201/charts/demo-hook
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/ingress.yaml
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/service.yaml
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/tests


!tree $wd_init

Create new (sub)chart in the *charts* dir
Creating work/helm-init/demo-helm-201/charts/demo-hook
work/helm-init
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    │   ├── demo-hook
    │   │   ├── Chart.yaml
    │   │   ├── charts
    │   │   ├── templates
    │   │   │   ├── NOTES.txt
    │   │   │   ├── _helpers.tpl
    │   │   │   ├── deployment.yaml
    │   │   │   └── serviceaccount.yaml
    │   │   └── values.yaml
    │   └── demo-subchart
    │       ├── Chart.yaml
    │       ├── charts
    │       ├── templates
    │       │   └── 03_configmap.yaml
    │       └── values.yaml
    ├── templates
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── ingress.yaml
    │   ├── sa.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    ├── values.pre.yaml
    ├── values.test.yaml
    └── values.yaml

10 directories, 21 files


---

* Action (04)
  * create 2 jobs for different hooks
  * install and upgrade

In [52]:
!echo "Install subchart for hook testing"

!echo "some OCP permissions adjustments..."
!oc adm policy add-scc-to-user anyuid -z release-demo-hook-dev

!helm upgrade --install release-demo-hook-dev $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

Install subchart for hook testing
Release "release-demo-hook-dev" does not exist. Installing it now.
NAME: release-demo-hook-dev
LAST DEPLOYED: Sat Feb  5 19:13:12 2022
NAMESPACE: demo-helm
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace demo-helm -l "app.kubernetes.io/name=demo-hook,app.kubernetes.io/instance=release-demo-hook-dev" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl --namespace demo-helm port-forward $POD_NAME 8080:80


In [57]:
!echo "Update subchart for hook testing now with job definitions"

!helm upgrade --install release-demo-hook-dev $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

Update subchart for hook testing now with job definitions
Release "release-demo-hook-dev" has been upgraded. Happy Helming!
NAME: release-demo-hook-dev
LAST DEPLOYED: Sat Feb  5 19:31:55 2022
NAMESPACE: demo-helm
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace demo-helm -l "app.kubernetes.io/name=demo-hook,app.kubernetes.io/instance=release-demo-hook-dev" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl --namespace demo-helm port-forward $POD_NAME 8080:80


---

* Action
  * current `pods` and `jobs`
  * events
  * clean-up

In [58]:
!echo "Delete subchart which triggers hook"

!helm delete release-demo-hook-dev -n demo-helm

Delete subchart which triggers hook
release "release-demo-hook-dev" uninstalled


---
---

## OCI Registry

Helm 3 provides the option to store helm charts not only on a HTTP server, but also in a OCI Registry. This allows the option to hold the artifacts (container image and helm charts) in the same registry.

### Testing

* Action
  * use a local registry
  * push a local helm chart
  * install from OCI registry

  

In [60]:
!echo "Run a local registry"

!docker run -dp 5000:5000 --restart=always --name registry registry

Run a local registry
Error: error creating container storage: the container name "registry" is already in use by "ef7052cabe86dca1e9b3c2b4fce12d377779ee2223edfa6918e793df65bc3cac". You have to remove that container to be able to reuse that name.: that name is already in use


In [66]:
!echo "Create helm package from the hook subchart"

!echo "...copy subchart in own dir for testing..."
!cp -r $wd_init/demo-helm-201/charts/demo-hook $wd_init/

!echo "...helm package..."
!helm package $wd_init/demo-hook --version 1.0.1 --destination $wd_init
!helm package $wd_init/demo-hook --version 1.0.2 --destination $wd_init

Create helm package from the hook subchart
...copy subchart in own dir for testing...
...helm package...
Successfully packaged chart and saved it to: work/helm-init/demo-hook-1.0.1.tgz
Successfully packaged chart and saved it to: work/helm-init/demo-hook-1.0.2.tgz


In [67]:
!echo "Push helm package to registry"

!echo "...helm registry login..."
!helm registry login -u testuser -p testpassword localhost:5000

!echo "...helm package push..."
!helm push $wd_init/demo-hook-1.0.1.tgz oci://localhost:5000/helm-charts
!helm push $wd_init/demo-hook-1.0.2.tgz oci://localhost:5000/helm-charts

Push helm package to registry
...helm registry login...
INFO[0000] Error logging in to endpoint, trying next endpoint  error="Get \"https://localhost:5000/v2/\": http: server gave HTTP response to HTTPS client"
Login Succeeded
...helm package push...
Pushed: localhost:5000/helm-charts/demo-hook:1.0.1
Digest: sha256:415e4318e0477edf396f3c5b076ac005173d1aaadb05dbe412999a5d8e047411
Pushed: localhost:5000/helm-charts/demo-hook:1.0.2
Digest: sha256:3896b2bdbb803737abadc97f857c61eacc5b542cd5a7beff087406a344b1adc2


In [70]:
!echo "Verify helm package in OCI registry"

!echo "...show chart details..."
!helm show chart oci://localhost:5000/helm-charts/demo-hook


Verify helm package in OCI registry
...show chart details...
apiVersion: v2
appVersion: 1.16.0
description: A Helm chart for Kubernetes
name: demo-hook
type: application
version: 1.0.2



In [72]:
!echo "Install (render) helm package directly from OCI registry"

!echo "...render existing chart from registry..."
!helm template release-demo-hook-oci oci://localhost:5000/helm-charts/demo-hook --version 1.0.1 --output-dir=work/out/oci

!echo "...try to render non-existing chart from registry..."
!helm template release-demo-hook-oci oci://localhost:5000/helm-charts/demo-hook --version 2.2.2 --output-dir=work/out/oci


Install (render) helm package directly from OCI registry
wrote work/out/oci/demo-hook/templates/serviceaccount.yaml
wrote work/out/oci/demo-hook/templates/deployment.yaml
wrote work/out/oci/demo-hook/templates/04_job.yaml
wrote work/out/oci/demo-hook/templates/04_job.yaml
wrote work/out/oci/demo-hook/templates/04_job.yaml

Error: failed to download "oci://localhost:5000/helm-charts/demo-hook" at version "2.2.2"


---
---

## Misc

Some additional details


### Internal/Release State

With Helm3 not Tiller exists anymore. All configuration and release state, including value files, is now stored in a `Secret` instead of a `ConfigMap`.


In [73]:
!echo "Install subchart"

!helm upgrade --install release-demo-hook-state $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

Install subchart
Release "release-demo-hook-state" does not exist. Installing it now.
I0206 10:47:50.174165   56123 request.go:665] Waited for 1.129503768s due to client-side throttling, not priority and fairness, request: GET:https://c113-e.eu-de.containers.cloud.ibm.com:30919/apis/discovery.k8s.io/v1beta1?timeout=32s
NAME: release-demo-hook-state
LAST DEPLOYED: Sun Feb  6 10:47:55 2022
NAMESPACE: demo-helm
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace demo-helm -l "app.kubernetes.io/name=demo-hook,app.kubernetes.io/instance=release-demo-hook-state" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl --namespace demo-helm port-forward $POD_NAME 8080:80


In [74]:
!echo "List all existing helm releases"

!helm list

List all existing helm releases
NAME                   	NAMESPACE	REVISION	UPDATED                             	STATUS  	CHART          	APP VERSION
release-demo-hook-state	demo-helm	1       	2022-02-06 10:47:55.639795 +0100 CET	deployed	demo-hook-0.1.0	1.16.0     


In [75]:
!echo "Print out release information from Helm Release Secret"

!oc get secret sh.helm.release.v1.release-demo-hook-state.v1 -o jsonpath='{.data.release}' | base64 --decode | base64 --decode | gunzip -c | jq

Print out release information from Helm Release Secret
{
  "name": "release-demo-hook-state",
  "info": {
    "first_deployed": "2022-02-06T10:47:55.639795+01:00",
    "last_deployed": "2022-02-06T10:47:55.639795+01:00",
    "deleted": "",
    "description": "Install complete",
    "status": "deployed",
    "notes": "1. Get the application URL by running these commands:\n  export POD_NAME=$(kubectl get pods --namespace demo-helm -l \"app.kubernetes.io/name=demo-hook,app.kubernetes.io/instance=release-demo-hook-state\" -o jsonpath=\"{.items[0].metadata.name}\")\n  echo \"Visit http://127.0.0.1:8080 to use your application\"\n  kubectl --namespace demo-helm port-forward $POD_NAME 8080:80\n"
  },
  "chart": {
    "metadata": {
      "name": "demo-hook",
      "version": "0.1.0",
      "description": "A Helm chart for Kubernetes",
      "apiVersion": "v2",
      "appVersion": "1.16.0",
      "type": "application"
    },
    "lock": null,
    "templates": [
      {
        "name": "template

### Helm Upgrade and force re-deployment

Helm determines the relevant changes and updates only the necessary resources. In case the content of a `ConfigMap` changed, but not the image version, a redeployment or restart of the deployment will be not triggered.
To trigger a redeployment a relation between `ConfigMap` content and `Deployment` is needed

```
kind: Deployment
spec:
  template:
    metadata:
      annotations:
        checksum/config: {{ include (print $.Template.BasePath "/configmap.yaml") . | sha256sum }}
```

The idea is, that the `Deployment` contains an annotation with a checksum of the related `ConfigMap`. Any changes in the content, results in an other `sha256sum`, results in an other annotation, which results in modified `Deployment` definition.

## Summary

This was a walkthrough for Helm 201 with covering topics like

* templating
* charts and subcharts
* OCI Registry

## References

* [Helm - Storage Provider - Secret instead ConfigMap](https://helm.sh/docs/topics/advanced/#storage-backends)
* [Helm - (OCI) Registry](https://helm.sh/docs/topics/registries/)
* [Helm - Best Practices](https://helm.sh/docs/chart_best_practices/conventions/)
* [Helm - Template Guide](https://helm.sh/docs/chart_template_guide/getting_started/)